In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
from scipy.signal import stft, get_window
from scipy.stats import entropy
import os
import pickle

### FUNCTIONS

In [2]:
# Función para cargar el archivo .set y obtener los datos y la información
def cargar_eeg(filepath):
    raw = mne.io.read_raw_eeglab(filepath, preload=True)
    return raw

# Función para añadir canales vacíos al objeto Raw de MNE
def add_empty_channels(raw, target_n_channels=16):
    n_channels = len(raw.ch_names)
    if n_channels < target_n_channels:
        mean_channel = np.mean(raw.get_data(), axis=0)
        for i in range(n_channels + 1, target_n_channels + 1):
            ch_name = f'EEG {i}'
            info = mne.create_info(ch_names=[ch_name], sfreq=raw.info['sfreq'], ch_types='eeg')
            new_raw = mne.io.RawArray(mean_channel[np.newaxis, :], info)
            raw.add_channels([new_raw], force_update_info=True)

In [3]:
# Define frequency bands
bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 14),
    'beta': (14, 31),
    'gamma': (31, 50)
}

# Compute Differential Entropy from STFT
def compute_de_from_stft(segment, fs=400):
    window_size = int(fs * 1)  # window size of 1 second
    de_features = []
    for band, (low, high) in bands.items():
        # Compute the STFT with a Hanning window
        f, t, Zxx = scipy.signal.stft(segment, fs=fs, window='hann', nperseg=window_size, noverlap=0)
        
        # Select frequencies within the band
        band_indices = np.where((f >= low) & (f <= high))[0]
        if len(band_indices) == 0:
            continue
        
        band_power = np.abs(Zxx[band_indices, :])**2
        band_power_mean = np.mean(band_power)
        
        # Compute differential entropy
        de = 0.5 * np.log(2 * np.pi * np.e * band_power_mean)
        de_features.append(de)
    return np.array(de_features).flatten()

# FEATURE EXTRACTION

In [4]:
input_dir = '.../7.EEGLAB_TrainingData_16Channels'
output_dir = '.../8.EEGLAB_TrainingData_Features'


# Listar todos los archivos .set en el directorio de entrada
for filename in os.listdir(input_dir):
    if filename.endswith('.set'):
        file_path = os.path.join(input_dir, filename)
        raw = cargar_eeg(file_path)
        add_empty_channels(raw)

        duration = 4
        epochs = mne.make_fixed_length_epochs(raw, duration=duration, preload=True, overlap=0)
        all_de_features = []

        for epoch_data in epochs.get_data():
            epoch_features = []
            for channel_data in epoch_data:
                de_features = compute_de_from_stft(channel_data, fs=raw.info['sfreq'])
                epoch_features.append(de_features)
            all_de_features.append(np.concatenate(epoch_features))

        all_de_features = np.array(all_de_features)

        # Guardar las características en un archivo
        output_file_path = os.path.join(output_dir, filename.replace('.set', '_features.pkl'))
        with open(output_file_path, 'wb') as f:
            pickle.dump(all_de_features, f)

        # Imprimir las dimensiones y un head de las primeras tres filas
        print(f"Processed {filename}: features saved to {output_file_path}")
        print("Shape of the features array:", all_de_features.shape)
        print(all_de_features[:1])

Not setting metadata
35 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 35 events and 1600 original time points ...
0 bad epochs dropped


C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed 10_1_10_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/8.EEGLAB_TrainingData_Features\10_1_10_preprocessed_features.pkl
Shape of the features array: (35, 80)
[[-14.84493715 -15.81362575 -16.30811985 -16.92733586 -17.19910199
  -14.81945127 -15.75412202 -16.2964294  -17.03127229 -17.33052251
  -15.04232725 -16.46043113 -16.6376381  -16.45479098 -16.42443108
  -15.25974262 -16.48811576 -16.67923913 -17.12970757 -17.20304109
  -15.47533111 -16.30689231 -16.46039902 -16.69826591 -16.46265399
  -15.89333165 -16.4479347  -16.67070184 -16.99276841 -16.9723082
  -15.73744698 -16.60202025 -16.97067425 -16.89002118 -16.59025785
  -15.2108502  -16.26203616 -16.05229138 -15.54492208 -15.29017988
  -15.87867142 -16.31591558 -16.29888137 -16.03962249 -15.95753625
  -16.03288009 -16.6563191  -16.79571836 -16.90763872 -16.63919892
  -14.7614387  -15.58544831 -15.88740333 -16.24920619 -16.32048563
  -15.1250275  -16.10850347 -16.02632118 -16.6197

C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed 10_1_11_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/8.EEGLAB_TrainingData_Features\10_1_11_preprocessed_features.pkl
Shape of the features array: (43, 80)
[[-13.57218455 -14.42948381 -15.25915233 -16.2787093  -16.74776614
  -13.58079337 -14.43907763 -15.18624991 -16.35746805 -17.11716721
  -14.50251045 -15.83773811 -15.87729979 -16.25612782 -16.4406198
  -14.30978817 -15.17403944 -15.50889137 -16.30980549 -16.83691885
  -14.19456136 -15.26152777 -15.53988662 -16.38833831 -16.75459002
  -14.24963459 -15.41067472 -15.70154259 -16.34325007 -16.48952385
  -14.92360194 -15.75430663 -16.22737332 -16.68938491 -16.89804723
  -14.99495387 -15.67692682 -15.24399085 -16.34956756 -16.82955331
  -14.85284687 -15.861205   -15.56526781 -15.50139735 -15.36226646
  -14.64224937 -15.61679512 -15.95939257 -16.09275183 -16.0687346
  -14.22536098 -15.0393388  -15.30617166 -16.01364161 -16.36430389
  -14.12663964 -15.22461064 -15.16270834 -15.99119

...

Processed 9_3_7_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/8.EEGLAB_TrainingData_Features\9_3_7_preprocessed_features.pkl
Shape of the features array: (21, 80)
[[-13.9021545  -14.45466683 -15.38752118 -16.13039498 -16.65580979
  -13.85291458 -14.42508688 -15.36642223 -16.11994353 -16.75767369
  -13.94938781 -14.79331829 -15.54724732 -15.99863021 -16.41962568
  -14.17154685 -14.65724073 -15.47616483 -15.89822067 -16.49093116
  -14.09314702 -14.53947368 -15.45326608 -15.95503204 -16.51400243
  -13.87534475 -14.61290384 -15.51610418 -16.14448653 -16.65265076
  -14.29202561 -14.84680139 -15.38055568 -15.17392896 -15.29331677
  -14.39929314 -14.80834348 -15.35500242 -15.50946332 -16.12101936
  -14.19790186 -14.78410664 -15.38424813 -15.6007981  -16.10473479
  -14.1930111  -15.06934426 -15.34563605 -15.06741887 -15.24200948
  -14.42805103 -14.84329432 -15.3328098  -15.5543065  -15.91535998
  -14.10358652 -14.35359966 -15.03088136 -15.6620605

C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed 9_3_8_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/8.EEGLAB_TrainingData_Features\9_3_8_preprocessed_features.pkl
Shape of the features array: (18, 80)
[[-13.73002866 -14.29127402 -15.11877481 -16.11225391 -16.81675009
  -13.71611669 -14.26317399 -15.1167324  -16.16469864 -16.95025008
  -13.83764699 -14.81319273 -15.46751064 -15.9680496  -16.25257661
  -13.86900059 -14.41680489 -15.18690753 -15.78508937 -16.19527273
  -13.87225276 -14.42732442 -15.16532484 -16.05996424 -16.5837542
  -13.73953589 -14.62984144 -15.4108942  -16.26054137 -16.63362551
  -14.25518807 -14.78590751 -15.40285256 -15.19194805 -15.47932598
  -14.31631824 -14.89359752 -15.27001149 -15.43784046 -16.37053096
  -13.9855375  -15.0428882  -15.39135179 -15.79376974 -16.33381804
  -13.99151727 -15.01935087 -15.42189591 -15.4787376  -15.73634031
  -14.08078646 -14.63575724 -15.42060772 -15.51238728 -15.76445101
  -13.88867851 -14.73917118 -15.11302812 -15.61195432

C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: Limited 2 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)
C:\Users\pablo\AppData\Local\Temp\ipykernel_2488\1625974722.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(filepath, preload=True)


Processed 9_3_9_preprocessed.set: features saved to C:/Users/pablo/OneDrive/Escritorio/UPV/TFG/Señales_Emilio/8.EEGLAB_TrainingData_Features\9_3_9_preprocessed_features.pkl
Shape of the features array: (57, 80)
[[-13.699314   -14.6621189  -15.6005202  -16.42484096 -16.71433325
  -13.64629544 -14.63128847 -15.54622446 -16.43354719 -16.79989156
  -13.90508044 -15.13089391 -15.85533991 -16.27271697 -16.34629063
  -13.74170001 -14.85015903 -15.57360189 -16.09549828 -16.34524358
  -13.59083559 -14.79728542 -15.4536652  -16.02896974 -16.46406059
  -13.76110689 -14.97675592 -15.7699216  -16.22680175 -16.71126228
  -14.05483689 -14.97128281 -15.33429754 -15.26192167 -15.4013453
  -14.32453579 -15.19435779 -15.45250698 -15.51398578 -16.23327983
  -14.12822261 -15.14424299 -15.49305485 -15.68070221 -16.18537262
  -14.44669796 -15.1763505  -15.88238836 -16.26625178 -16.6769127
  -13.81754316 -15.04048326 -15.6894515  -16.0743851  -16.32827474
  -13.68626352 -14.74162196 -15.05257179 -15.62803007 

# AÑADIR ETIQUETAS. SEED V

In [5]:
# AÑADIR ETIQUETAS. #

# Definir el diccionario de etiquetas basado en la sesión y el trial
labels = {
    1: ['Happy', 'Fear', 'Neutral', 'Sad', 'Disgust', 'Happy', 'Fear', 'Neutral', 'Sad', 'Disgust', 'Happy', 'Fear', 'Neutral', 'Sad', 'Disgust'],
    2: ['Sad', 'Fear', 'Neutral', 'Disgust', 'Happy', 'Happy', 'Disgust', 'Neutral', 'Sad', 'Fear', 'Neutral', 'Happy', 'Fear', 'Sad', 'Disgust'],
    3: ['Sad', 'Fear', 'Neutral', 'Disgust', 'Happy', 'Happy', 'Disgust', 'Neutral', 'Sad', 'Fear', 'Neutral', 'Happy', 'Fear', 'Sad', 'Disgust']
}

# Rutas de los archivos
ruta_archivos = '.../8.EEGLAB_TrainingData_Features'
ruta_guardado = r'...\9.EEGLAB_TrainingData_Labeled'

# Crear la ruta de guardado si no existe
os.makedirs(ruta_guardado, exist_ok=True)

# Función para procesar los archivos
def agregar_etiqueta_a_archivos(ruta_lectura, ruta_escritura):
    for archivo in os.listdir(ruta_lectura):
        if archivo.endswith('.pkl'):
            # Extraer números de sesión y trial del nombre del archivo
            partes = archivo.split('_')
            if len(partes) >= 4:
                sesion = int(partes[1])  # Asumiendo que el segundo elemento es la sesión
                trial = int(partes[2])  # Asumiendo que el tercer elemento es el trial
                
                # Cargar el archivo .pkl
                with open(os.path.join(ruta_lectura, archivo), 'rb') as f:
                    data = pickle.load(f)
                
                # Obtener la etiqueta correspondiente
                etiqueta = labels[sesion][trial - 1]
                
                # Crear una columna de etiquetas
                etiqueta_columna = np.full((data.shape[0], 1), etiqueta, dtype=object)
                
                # Concatenar la columna de etiquetas al array de datos
                data_con_etiqueta = np.hstack((data, etiqueta_columna))
                
                # Guardar el nuevo archivo con la etiqueta añadida
                nuevo_nombre_archivo = os.path.join(ruta_escritura, archivo)
                with open(nuevo_nombre_archivo, 'wb') as f:
                    pickle.dump(data_con_etiqueta, f)
                print(f"Archivo procesado y guardado: {nuevo_nombre_archivo}")

# Ejecutar la función
agregar_etiqueta_a_archivos(ruta_archivos, ruta_guardado)


Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\9.EEGLAB_TrainingData_Labeled\10_1_10_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\9.EEGLAB_TrainingData_Labeled\10_1_11_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\9.EEGLAB_TrainingData_Labeled\10_1_12_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\9.EEGLAB_TrainingData_Labeled\10_1_13_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\9.EEGLAB_TrainingData_Labeled\10_1_14_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\9.EEGLAB_TrainingData_Labeled\10_1_15_preprocessed_features.pkl
Archivo procesado y guardado: C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\

In [6]:
#COMPROBACION ETIQUETAS
import pandas as pd
import numpy as np
import pickle

def mostrar_valores_unicos_ultima_columna(ruta_archivo):
    # Abrir el archivo pickle y cargar el contenido
    with open(ruta_archivo, 'rb') as file:
        data = pickle.load(file)
    
    # Verificar si el contenido es un array de NumPy
    if isinstance(data, np.ndarray):
        # Asumir que la última columna contiene las etiquetas
        ultima_columna = data[:, -1]  # Extraer la última columna del array
        valores_unicos = np.unique(ultima_columna)  # Encontrar valores únicos
        
        # Mostrar los valores únicos
        print("Valores únicos en la última columna:")
        print(valores_unicos)
    else:
        print("El contenido no es un array de NumPy.")

# Uso del método
ruta_pickle = r'C:\Users\pablo\OneDrive\Escritorio\UPV\TFG\Señales_Emilio\9.EEGLAB_TrainingData_Labeled\16_1_5_preprocessed_features.pkl'
mostrar_valores_unicos_ultima_columna(ruta_pickle)

Valores únicos en la última columna:
['Disgust']


# CONCATENAR Y EXPORTAR CSV

In [7]:
#ETIQUETAS A NUMERICO Y CONCATENAR

import tempfile

# Ruta de entrada de los archivos
ruta_entrada = r'...\9.EEGLAB_TrainingData_Labeled'

# Usar una ruta temporal para la salida
ruta_salida = tempfile.gettempdir()
nombre_archivo = 'training_combinado.csv'
ruta_archivo_salida = os.path.join(ruta_salida, nombre_archivo)

# Diccionario de mapeo de etiquetas
label_mapping = {
    'Disgust': 0,
    'Fear': 1,
    'Sad': 2,
    'Neutral': 3,
    'Happy': 4
}

def procesar_archivos_y_combinar(ruta_entrada):
    dataframes = []
    
    for archivo in os.listdir(ruta_entrada):
        if archivo.endswith('.pkl'):
            ruta_completa = os.path.join(ruta_entrada, archivo)
            with open(ruta_completa, 'rb') as f:
                data = pickle.load(f)
            
            # Asumimos que las etiquetas están en la última columna
            etiquetas_numericas = np.vectorize(label_mapping.get)(data[:, -1].astype(str))
            
            # Reemplazamos la última columna con las nuevas etiquetas numéricas
            data[:, -1] = etiquetas_numericas
            
            # Convertimos el array a DataFrame
            df = pd.DataFrame(data)
            dataframes.append(df)
    
    # Combinar todos los DataFrames en uno solo
    df_combinado = pd.concat(dataframes, ignore_index=True)
    
    # Renombrar la última columna a 'Label'
    df_combinado.rename(columns={df_combinado.columns[-1]: 'Label'}, inplace=True)
    
    return df_combinado

# Llamar a la función y obtener el DataFrame combinado
df_final = procesar_archivos_y_combinar(ruta_entrada)

# Guardar el DataFrame final como CSV
df_final.to_csv(ruta_archivo_salida, index=False)

print("DataFrame combinado guardado en:", ruta_archivo_salida)

DataFrame combinado guardado en: C:\Users\pablo\AppData\Local\Temp\training_combinado.csv


In [8]:
#COMPROBACION CSV

# Ruta al archivo CSV generado
ruta_csv = r'...\training_combinado.csv'

# Cargar el CSV en un DataFrame
df = pd.read_csv(ruta_csv)

# Obtener las dimensiones del DataFrame
dimensiones = df.shape
print("Dimensiones del DataFrame:", dimensiones)

# Obtener valores únicos de la última columna (asumimos que se llama 'Label')
valores_unicos = df['Label'].unique()
print("Valores únicos en la última columna 'Label':", valores_unicos)

Dimensiones del DataFrame: (29168, 81)
Valores únicos en la última columna 'Label': [0 4 1 3 2]


In [9]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,Label
0,-11.333308,-12.135897,-12.675679,-13.331359,-13.727481,-11.388006,-12.203800,-12.781678,-13.570344,-13.952715,...,-12.851682,-12.990042,-13.591615,-13.369385,-11.980309,-13.026783,-13.332436,-13.859360,-14.196689,0
1,-11.611512,-12.404390,-12.674059,-13.507144,-13.513519,-11.589014,-12.513252,-12.748351,-13.565917,-13.821977,...,-12.534513,-12.704353,-13.323224,-13.286100,-12.160843,-12.809061,-12.956309,-13.544440,-14.059632,0
2,-10.388916,-11.414412,-12.541039,-13.361516,-13.684494,-10.389087,-11.445266,-12.581468,-13.471382,-14.018159,...,-12.329777,-13.150360,-13.447942,-13.453678,-11.306958,-12.296105,-13.229788,-13.731503,-14.184666,0
3,-10.851834,-12.056035,-12.683115,-13.415757,-13.661156,-10.854513,-12.088723,-12.745659,-13.567411,-13.868283,...,-12.818469,-13.218444,-13.374892,-13.599352,-11.669535,-12.974209,-13.166486,-13.686460,-14.273101,0
4,-10.643435,-11.666463,-12.403969,-13.453758,-13.644428,-10.630437,-11.637024,-12.407587,-13.525638,-13.846568,...,-12.422949,-12.736410,-13.453645,-13.430150,-11.529921,-12.578192,-13.005448,-13.662122,-14.129633,0
